In [140]:
from copy import deepcopy
import re
import numpy as np
import pandas as pd
import matplotlib as plt

In [5]:
colnames = ['sentence_id', 'text', 'pair_id', 'drug1_id', 'drug1', 'drug1_type', 'drug2_id', 'drug2', 'drug2_type', 'ddi', 'ddi_type']
train_data = pd.read_csv('./ddi_train.csv', header = None, names = colnames)
test_data = pd.read_csv('./ddi_test.csv', header = None, names = colnames)

In [18]:
train_data.head()

,sentence_id,text,pair_id,drug1_id,drug1,drug1_type,drug2_id,drug2,drug2_type,ddi,ddi_type
0,DDI-DrugBank.d519.s3,laboratory tests response to plenaxis should b...,DDI-DrugBank.d519.s3.p0,DDI-DrugBank.d519.s3.e0,plenaxis,brand,DDI-DrugBank.d519.s3.e1,testosterone,drug,False,NaN
1,DDI-DrugBank.d297.s1,population pharmacokinetic analyses revealed t...,DDI-DrugBank.d297.s1.p0,DDI-DrugBank.d297.s1.e0,mtx,drug,DDI-DrugBank.d297.s1.e1,nsaids,group,False,NaN
2,DDI-DrugBank.d297.s1,population pharmacokinetic analyses revealed t...,DDI-DrugBank.d297.s1.p1,DDI-DrugBank.d297.s1.e0,mtx,drug,DDI-DrugBank.d297.s1.e2,corticosteroids,group,False,NaN
3,DDI-DrugBank.d297.s1,population pharmacokinetic analyses revealed t...,DDI-DrugBank.d297.s1.p2,DDI-DrugBank.d297.s1.e0,mtx,drug,DDI-DrugBank.d297.s1.e3,tnf blocking agents,group,False,NaN
4,DDI-DrugBank.d297.s1,population pharmacokinetic analyses revealed t...,DDI-DrugBank.d297.s1.p3,DDI-DrugBank.d297.s1.e0,mtx,drug,DDI-DrugBank.d297.s1.e4,abatacept,drug,False,NaN


In [86]:
len(drug_list)

2242

In [200]:
#mini set to test functions with
mini_set = deepcopy(train_data.loc[1535:1545])

In [214]:
drug1_list = np.unique(train_data['drug1'])
drug2_list = np.unique(train_data['drug2'])
drug_list = np.unique(np.concatenate([drug1_list, drug2_list]))
drug_list = np.delete(drug_list, np.where(drug_list == 'drug')[0][0])


def anonymize_drugs(data):
    '''replaces the drug mentions in the sentences with drug1, drug2, or drug0. The drug pair of interest is
    replaced with drug1 and drug2 while other drug mentions in the sentence that are not part of the pair
    are replaced with drug0
    
    Example:
    
    laboratory tests response to plenaxis should be monitored by measuring serum total testosterone 
    concentrations just prior to administration on day 29 and every 8 weeks thereafter.
    
    If the pair of interest is plenaxis and testosterone, this sentence becomes:
    
    laboratory tests response to drug1 should be monitored by measuring serum total drug2 
    concentrations just prior to administration on day 29 and every 8 weeks thereafter.
    '''
    sentences = data['text']
    drug1_list = data['drug1']
    drug2_list = data['drug2']
    drug_list = np.unique(np.concatenate([drug1_list, drug2_list]))
    if 'drug' in drug_list:
        drug_list = np.delete(drug_list, np.where(drug_list == 'drug')[0][0])
    for i in range(len(sentences)):
        sentence = sentences.iloc[i]
        print('original sentence')
        print(sentence)
        print('----')
        regex_drug1 = re.compile(f'\s{drug1_list.iloc[i]}[\s,.;:]')
        regex_drug2 = re.compile(f'\s{drug2_list.iloc[i]}[\s,.;:]')
        punctuation_list = [' ', ',', '.', ':', ';']
        
        
        for punctuation in punctuation_list:
            pattern = re.compile(f'\s{drug1_list.iloc[i]}{punctuation}')
            if pattern.search(sentence):
                sentence = regex_drug1.sub(f' drug1{punctuation}', sentence, count = 1)
                break

        for punctuation in punctuation_list:
            pattern = re.compile(f'\s{drug2_list.iloc[i]}{punctuation}')
            if pattern.search(sentence):
                sentence = regex_drug2.sub(f' drug2{punctuation}', sentence, count = 1)
                break
                
        for drug in drug_list:
            regex_drug0 = re.compile(f'\s{drug}[\s,.;:]')
            for punctuation in punctuation_list:
                pattern = re.compile(f'\s{drug}{punctuation}')
                if pattern.search(sentence):
                    sentence = regex_drug0.sub(f' drug0{punctuation}', sentence, count = 1)
                    break
            
            
            
            
            
            
        print(sentence)
        print('---')
    return sentence
            
    
    

In [215]:
anonymize_drugs(mini_set)

original sentence
imidazoles (e. g., ketoconazole, miconazole, clotrimazole, fluconazole, etc.): in vitro and animal studies with the combination of amphotericin b and imidazoles suggest that imidazoles may induce fungal resistance to amphotericin b.
----
imidazoles (e. g., drug0, drug2, drug0, drug0, etc.): in vitro and animal studies with the combination of drug0 and drug1 suggest that drug0 may induce fungal resistance to amphotericin b.
---
original sentence
imidazoles (e. g., ketoconazole, miconazole, clotrimazole, fluconazole, etc.): in vitro and animal studies with the combination of amphotericin b and imidazoles suggest that imidazoles may induce fungal resistance to amphotericin b.
----
imidazoles (e. g., drug0, drug0, drug2, drug0, etc.): in vitro and animal studies with the combination of drug0 and drug1 suggest that drug0 may induce fungal resistance to amphotericin b.
---
original sentence
imidazoles (e. g., ketoconazole, miconazole, clotrimazole, fluconazole, etc.): in vi

'imidazoles (e. g., drug1, drug0, drug0, drug0, etc.): in vitro and animal studies with the combination of drug2 and drug0 suggest that imidazoles may induce fungal resistance to drug0.'

In [158]:
for text in mini_set['text']:
    print(text)

laboratory tests response to plenaxis should be monitored by measuring serum total testosterone concentrations just prior to administration on day 29 and every 8 weeks thereafter.
population pharmacokinetic analyses revealed that mtx, nsaids, corticosteroids, and tnf blocking agents did not influence abatacept clearance.
population pharmacokinetic analyses revealed that mtx, nsaids, corticosteroids, and tnf blocking agents did not influence abatacept clearance.
population pharmacokinetic analyses revealed that mtx, nsaids, corticosteroids, and tnf blocking agents did not influence abatacept clearance.
population pharmacokinetic analyses revealed that mtx, nsaids, corticosteroids, and tnf blocking agents did not influence abatacept clearance.
population pharmacokinetic analyses revealed that mtx, nsaids, corticosteroids, and tnf blocking agents did not influence abatacept clearance.


In [106]:
train_data.iloc[len(train_data) -1]


sentence_id                                  DDI-MedLine.d113.s5
text           the results raise the possibility that the eth...
pair_id                                   DDI-MedLine.d113.s5.p0
drug1_id                                  DDI-MedLine.d113.s5.e0
drug1                                                    alcohol
drug1_type                                                  drug
drug2_id                                  DDI-MedLine.d113.s5.e1
drug2                               3-hydroxy-1,4-benzodiazepine
drug2_type                                                 group
ddi                                                         True
ddi_type                                               mechanism
Name: 27874, dtype: object

731

In [113]:
def anonymize_sentence(sentence, drug1, drug2, drug_list):
    s = sentence.replace(drug1, 'drug1')
    print(s)
    s = s.replace(drug2, 'drug2')
    for drug in drug_list:
        s = s.replace(drug, 'drug0')
    return s

In [125]:
train_data[train_data['drug2'] == 'drug'].iloc[0]

sentence_id                                 DDI-DrugBank.d99.s23
text           increases in plasma levels of tricyclic antide...
pair_id                                  DDI-DrugBank.d99.s23.p2
drug1_id                                 DDI-DrugBank.d99.s23.e0
drug1                                  tricyclic antidepressants
drug1_type                                                  drug
drug2_id                                 DDI-DrugBank.d99.s23.e3
drug2                                                       drug
drug2_type                                                  drug
ddi                                                        False
ddi_type                                                     NaN
Name: 1509, dtype: object

In [114]:
sentence = 'the results raise the possibility that the ethanolysis reaction may occur in the stomach of people who consume alcohol and 3-hydroxy-1,4-benzodiazepine on a regular basis. '
drug1 = 'alcohol'
drug2 = '3-hydroxy-1,4-benzodiazepine'

y = anonymize_sentence(sentence, drug1, drug2, drug_list = drug_list)
print(y)

the results raise the possibility that the ethanolysis reaction may occur in the stomach of people who consume drug1 and 3-hydroxy-1,4-benzodiazepine on a regular basis. 
the results raise the possibility that the drug0ysis reaction may occur in the stomach of people who drug00nsume drug01 and drug02 on a regular basis. 


In [92]:
'hello mary'.replace('hello mary', 'wee')

'wee'

In [72]:
anonymized.loc[0]['text']

'laboratory tests response to plenaxis should be monitored by measuring serum total testosterone concentrations just prior to administration on day 29 and every 8 weeks thereafter.'

In [58]:
mini_set

,sentence_id,text,pair_id,drug1_id,drug1,drug1_type,drug2_id,drug2,drug2_type,ddi,ddi_type
0,DDI-DrugBank.d519.s3,laboratory tests response to plenaxis should b...,DDI-DrugBank.d519.s3.p0,DDI-DrugBank.d519.s3.e0,plenaxis,brand,DDI-DrugBank.d519.s3.e1,testosterone,drug,False,NaN
1,DDI-DrugBank.d297.s1,population pharmacokinetic analyses revealed t...,DDI-DrugBank.d297.s1.p0,DDI-DrugBank.d297.s1.e0,mtx,drug,DDI-DrugBank.d297.s1.e1,nsaids,group,False,NaN
2,DDI-DrugBank.d297.s1,population pharmacokinetic analyses revealed t...,DDI-DrugBank.d297.s1.p1,DDI-DrugBank.d297.s1.e0,mtx,drug,DDI-DrugBank.d297.s1.e2,corticosteroids,group,False,NaN
3,DDI-DrugBank.d297.s1,population pharmacokinetic analyses revealed t...,DDI-DrugBank.d297.s1.p2,DDI-DrugBank.d297.s1.e0,mtx,drug,DDI-DrugBank.d297.s1.e3,tnf blocking agents,group,False,NaN
4,DDI-DrugBank.d297.s1,population pharmacokinetic analyses revealed t...,DDI-DrugBank.d297.s1.p3,DDI-DrugBank.d297.s1.e0,mtx,drug,DDI-DrugBank.d297.s1.e4,abatacept,drug,False,NaN


In [48]:
len(train_data)

27875